In [ ]:
import sys
!{sys.executable} -m pip install torch torchvision torchaudio
!{sys.executable} -m pip install opencv-python pillow matplotlib pandas numpy scikit-learn
!{sys.executable} -m pip install seaborn tqdm

In [ ]:
import torch
import torchvision
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import sklearn

print("All libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")  
print(f"MPS available: {torch.backends.mps.is_available()}") 
print(f"Pandas version: {pd.__version__}")

x = torch.tensor([1, 2, 3])
print(f"Tensor test: {x + 1}")

In [ ]:
import pandas as pd

train_csv_path = os.path.join(dataset_path, 'Train', 'sliders.csv')
validation_csv_path = os.path.join(dataset_path, 'Validation', 'sliders_input.csv')

train_df = pd.read_csv(train_csv_path)
print(f"Shape: {train_df.shape}")  
print(train_df.head())
print(train_df.columns.tolist())
print(train_df.info())

print("\n" + "="*50)

validation_df = pd.read_csv(validation_csv_path)
print(f"Shape: {validation_df.shape}")
print(validation_df.head())
print(validation_df.columns.tolist())

In [ ]:
print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(validation_df)}")

print(f"Temperature range: {train_df['Temperature'].min()} to {train_df['Temperature'].max()}")
print(f"Tint range: {train_df['Tint'].min()} to {train_df['Tint'].max()}")

print(f"- Aperture: {train_df['aperture'].min()} to {train_df['aperture'].max()}")
print(f"- Focal Length: {train_df['focalLength'].min()} to {train_df['focalLength'].max()}")
print(f"- ISO: {train_df['isoSpeedRating'].min()} to {train_df['isoSpeedRating'].max()}")

print(f"Current Temp (As Shot WB): {train_df['currTemp'].min()} to {train_df['currTemp'].max()}")
print(f"Current Tint (As Shot WB): {train_df['currTint'].min()} to {train_df['currTint'].max()}")

train_images_path = os.path.join(dataset_path, 'Train', 'images')
validation_images_path = os.path.join(dataset_path, 'Validation', 'images')

print(f"Train images folder exists: {os.path.exists(train_images_path)}")
print(f"Validation images folder exists: {os.path.exists(validation_images_path)}")

if os.path.exists(train_images_path):
    train_images = os.listdir(train_images_path)
    print(f"Number of train images: {len(train_images)}")
    print(f"First 3 train images: {train_images[:3]}")

In [ ]:
import sys
!{sys.executable} -m pip install "numpy<2" --force-reinstall

In [ ]:
import numpy as np
import pandas as pd
import torch

print("NumPy version:", np.__version__)
print("Pandas version:", pd.__version__)
print("PyTorch version:", torch.__version__)

# Quick test
print("Basic test:", np.array([1, 2, 3]) + 1)

In [ ]:
class WhiteBalanceLoss(nn.Module):
    def __init__(self, temp_weight=1.0, tint_weight=1.0):
        super(WhiteBalanceLoss, self).__init__()
        self.temp_weight = temp_weight
        self.tint_weight = tint_weight
        
    def forward(self, predictions, targets):
        pred_temp, pred_tint = predictions[:, 0], predictions[:, 1]
        true_temp, true_tint = targets[:, 0], targets[:, 1]
        
        temp_weights = 1.0 / torch.sqrt(true_temp / 1000.0 + 1e-6)
        temp_loss = torch.mean(temp_weights * (pred_temp - true_temp) ** 2)
        
        tint_loss = torch.mean((pred_tint - true_tint) ** 2)
        
        total_loss = self.temp_weight * temp_loss + self.tint_weight * tint_loss
        
        return total_loss

criterion = WhiteBalanceLoss()

test_predictions = torch.tensor([[4150.0, 2.0], [3000.0, -5.0]])
test_targets = torch.tensor([[4200.0, 1.0], [3100.0, -3.0]])

loss = criterion(test_predictions, test_targets)
print(f"Loss function test: {loss.item():.4f}")

In [ ]:
batch_size = 16  

train_loader = DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    shuffle=True,
    num_workers=0  
)

print(f"Training DataLoader created:")
print(f"- Batch size: {batch_size}")
print(f"- Total batches: {len(train_loader)}")
print(f"- Samples per epoch: {len(train_dataset)}")

for images, metadata, targets in train_loader:
    print(f"Batch test successful:")
    print(f"- Images shape: {images.shape}")
    print(f"- Metadata shape: {metadata.shape}") 
    print(f"- Targets shape: {targets.shape}")
    break 

In [ ]:
import pandas as pd
import torch
import torch.nn as nn

dataset_path = 'enter your data path'
train_df = pd.read_csv(os.path.join(dataset_path, 'Train', 'sliders.csv'))

def scale_targets(targets):
    """Scale Temperature and Tint to reasonable ranges"""
    temp_scaled = targets[:, 0] / 1000.0  # Scale Temperature from 2000-50000 to 2-50
    tint_scaled = targets[:, 1] / 10.0    # Scale Tint from -150-150 to -15-15
    return torch.stack([temp_scaled, tint_scaled], dim=1)

def unscale_predictions(predictions):
    """Convert scaled predictions back to original range"""
    temp_original = predictions[:, 0] * 1000.0
    tint_original = predictions[:, 1] * 10.0
    return torch.stack([temp_original, tint_original], dim=1)

test_targets = torch.tensor([[4150.0, 2.0], [2000.0, -50.0]])
scaled = scale_targets(test_targets)
unscaled = unscale_predictions(scaled)

print(f"Scaling test:")
print(f"Original: {test_targets[0].numpy()}")
print(f"Scaled: {scaled[0].numpy()}")
print(f"Unscaled: {unscaled[0].numpy()}")

print(f"\nDATA DISTRIBUTION ANALYSIS:")
print(f"Temperature - Min: {train_df['Temperature'].min()}, Max: {train_df['Temperature'].max()}")
print(f"Tint - Min: {train_df['Tint'].min()}, Max: {train_df['Tint'].max()}")
print(f"As Shot Temp - Min: {train_df['currTemp'].min()}, Max: {train_df['currTemp'].max()}")

In [ ]:
class ImprovedWhiteBalanceModel(nn.Module):
    def __init__(self):
        super(ImprovedWhiteBalanceModel, self).__init__()
        
        self.backbone = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
        self.backbone.classifier = nn.Identity()
        image_feature_dim = 1280
        
        self.metadata_fc = nn.Sequential(
            nn.Linear(8, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        
        self.combined_fc = nn.Sequential(
            nn.Linear(image_feature_dim + 128, 512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 2),
            nn.Tanh()  
        )
        
    def forward(self, image, metadata):
        image_features = self.backbone(image)
        metadata_features = self.metadata_fc(metadata)
        combined_features = torch.cat([image_features, metadata_features], dim=1)
        output = self.combined_fc(combined_features)
        return output

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
improved_model = ImprovedWhiteBalanceModel().to(device)
improved_optimizer = torch.optim.AdamW(improved_model.parameters(), lr=0.0005)  

improved_model.eval()
with torch.no_grad():
    from PIL import Image
    import torchvision.transforms as transforms
    
    test_transform = transforms.Compose([
        transforms.Resize((384, 384)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    test_input = torch.randn(1, 3, 384, 384).to(device)
    test_metadata = torch.randn(1, 8).to(device)
    
    test_pred = improved_model(test_input, test_metadata)
    print(f"Improved model output range: {test_pred.min().item():.3f} to {test_pred.max().item():.3f}")
    print(f"After unscaling: Temp ~{test_pred[0,0].item()*1000:.0f}K, Tint ~{test_pred[0,1].item()*10:.0f}")

In [ ]:
import time
from tqdm import tqdm

class ScaledWhiteBalanceLoss(nn.Module):
    def __init__(self):
        super(ScaledWhiteBalanceLoss, self).__init__()
        
    def forward(self, predictions, targets):
        pred_temp, pred_tint = predictions[:, 0], predictions[:, 1]
        true_temp, true_tint = targets[:, 0], targets[:, 1]
        
        temp_weights = 1.0 / torch.sqrt(true_temp + 1.0)  
        temp_loss = torch.mean(temp_weights * (pred_temp - true_temp) ** 2)
        
        tint_loss = torch.mean((pred_tint - true_tint) ** 2)
        
        return temp_loss + tint_loss

improved_model.train()
criterion = ScaledWhiteBalanceLoss()

num_epochs = 3  
train_losses = []

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    start_time = time.time()
    
    for batch_idx, (images, metadata, targets) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")):
        images = images.to(device)
        metadata = metadata.to(device)
        targets = targets.to(device)
        
        scaled_targets = scale_targets(targets)
        
        improved_optimizer.zero_grad()
        predictions = improved_model(images, metadata)
        loss = criterion(predictions, scaled_targets)
        
        loss.backward()
        improved_optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_loss)
    
    print(f"Epoch {epoch+1} Complete:")
    print(f"- Average Loss: {avg_loss:.4f}")
    print(f"- Time: {time.time() - start_time:.1f}s")
    
    improved_model.eval()
    with torch.no_grad():
        test_image, test_metadata, test_target = train_dataset[0]
        test_image = test_image.unsqueeze(0).to(device)
        test_metadata = test_metadata.unsqueeze(0).to(device)
        
        prediction = improved_model(test_image, test_metadata)
        unscaled_pred = unscale_predictions(prediction)
        
        print(f"- Sample Prediction:")
        print(f"Predicted: Temp {unscaled_pred[0,0].item():.0f}K, Tint {unscaled_pred[0,1].item():.0f}")
        print(f"Actual:    Temp {test_target[0].item():.0f}K, Tint {test_target[1].item():.0f}")
        print(f"Error:     Temp {abs(unscaled_pred[0,0].item() - test_target[0].item()):.0f}K")

print(f"\nTRAINING COMPLETE!")
print(f"Final losses: {train_losses}")

if len(train_losses) > 1 and train_losses[-1] < train_losses[0]:
    print("SUCCESS: Model is learning! Loss decreased.")
else:
    print("Loss not decreasing significantly - may need more epochs or tuning")

In [ ]:
improved_model.eval()
with torch.no_grad():
    print("\nTesting on multiple samples:")
    for i in range(3):
        test_image, test_metadata, test_target = train_dataset[i]
        test_image = test_image.unsqueeze(0).to(device)
        test_metadata = test_metadata.unsqueeze(0).to(device)
        
        prediction = improved_model(test_image, test_metadata)
        unscaled_pred = unscale_predictions(prediction)
        
        print(f"Sample {i}:")
        print(f"  As Shot WB: Temp {test_metadata[0,0].item():.0f}K, Tint {test_metadata[0,1].item():.0f}")
        print(f"  Predicted:  Temp {unscaled_pred[0,0].item():.0f}K, Tint {unscaled_pred[0,1].item():.0f}")
        print(f"  Actual:     Temp {test_target[0].item():.0f}K, Tint {test_target[1].item():.0f}")
        print()

def calculate_adjustments(targets, metadata):
    temp_adjustment = (targets[:, 0] - metadata[:, 0]) / 1000.0  # Scale adjustment
    tint_adjustment = (targets[:, 1] - metadata[:, 1]) / 10.0    # Scale adjustment
    return torch.stack([temp_adjustment, tint_adjustment], dim=1)

def apply_adjustments(predictions, metadata):
    temp_absolute = predictions[:, 0] * 1000.0 + metadata[:, 0]
    tint_absolute = predictions[:, 1] * 10.0 + metadata[:, 1]
    return torch.stack([temp_absolute, tint_absolute], dim=1)

test_targets = torch.tensor([[4150.0, 2.0], [3000.0, -5.0]])
test_metadata = torch.tensor([[6317.0, 4.0], [4000.0, 10.0]])  # As Shot WB

adjustments = calculate_adjustments(test_targets, test_metadata)
reconstructed = apply_adjustments(adjustments, test_metadata)

print("Adjustment approach test:")
print(f"Original: {test_targets[0].numpy()}")
print(f"As Shot:  {test_metadata[0].numpy()}")
print(f"Adjustment: {adjustments[0].numpy()}")
print(f"Reconstructed: {reconstructed[0].numpy()}")

In [ ]:
class AdjustmentModel(nn.Module):
    def __init__(self):
        super(AdjustmentModel, self).__init__()
        
        self.backbone = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
        self.backbone.classifier = nn.Identity()
        image_feature_dim = 1280
        
        self.metadata_fc = nn.Sequential(
            nn.Linear(8, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU()
        )
        
        self.combined_fc = nn.Sequential(
            nn.Linear(image_feature_dim + 64, 512),  
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 2),  
            nn.Tanh()  
        )
        
    def forward(self, image, metadata):
        image_features = self.backbone(image)
        metadata_features = self.metadata_fc(metadata)
        combined = torch.cat([image_features, metadata_features], dim=1)
        adjustments = self.combined_fc(combined)
        return adjustments

print("Creating adjustment-based model...")
adjustment_model = AdjustmentModel().to(device)
adjustment_optimizer = torch.optim.Adam(adjustment_model.parameters(), lr=0.001)

print(f"Model created with {sum(p.numel() for p in adjustment_model.parameters()):,} parameters")

adjustment_model.eval()
with torch.no_grad():
    test_image, test_metadata, test_target = train_dataset[0]
    test_image = test_image.unsqueeze(0).to(device)
    test_metadata = test_metadata.unsqueeze(0).to(device)
    
    test_pred = adjustment_model(test_image, test_metadata)
    print(f"Adjustment model output: {test_pred[0].cpu().numpy()}")
    
    absolute_pred = apply_adjustments(test_pred, test_metadata)
    print(f"Converted to absolute: Temp {absolute_pred[0,0].item():.0f}K, Tint {absolute_pred[0,1].item():.0f}")
    print(f"Actual target: Temp {test_target[0].item():.0f}K, Tint {test_target[1].item():.0f}")

In [ ]:
def adjustment_loss(predictions, targets, metadata):
    target_adjustments = calculate_adjustments(targets, metadata)
    
    loss = F.mse_loss(predictions, target_adjustments)
    return loss

adjustment_model.train()
batch_images, batch_metadata, batch_targets = next(iter(train_loader))

batch_images = batch_images.to(device)
batch_metadata = batch_metadata.to(device)
batch_targets = batch_targets.to(device)

adjustment_optimizer.zero_grad()
predicted_adjustments = adjustment_model(batch_images, batch_metadata)
loss = adjustment_loss(predicted_adjustments, batch_targets, batch_metadata)

print(f"Initial batch loss: {loss.item():.4f}")

loss.backward()
adjustment_optimizer.step()

print("Model can train successfully!")
print(f"Gradient flow: {any(p.grad is not None and p.grad.abs().sum() > 0 for p in adjustment_model.parameters())}")

adjustment_model.eval()
with torch.no_grad():
    test_pred = adjustment_model(test_image, test_metadata)
    absolute_pred = apply_adjustments(test_pred, test_metadata)
    print(f"After one step - Predicted: Temp {absolute_pred[0,0].item():.0f}K, Tint {absolute_pred[0,1].item():.0f}")

In [ ]:
import time
from tqdm import tqdm

adjustment_model.train()
train_losses = []
num_epochs = 5  # Let's do 5 epochs

for epoch in range(num_epochs):
    epoch_loss = 0.0
    start_time = time.time()
    
    for batch_idx, (images, metadata, targets) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")):
        images = images.to(device)
        metadata = metadata.to(device)
        targets = targets.to(device)
        
        adjustment_optimizer.zero_grad()
        predicted_adjustments = adjustment_model(images, metadata)
        loss = adjustment_loss(predicted_adjustments, targets, metadata)
        
        loss.backward()
        adjustment_optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_loss)
    
    print(f"Epoch {epoch+1} Complete:")
    print(f"- Average Loss: {avg_loss:.4f}")
    print(f"- Time: {time.time() - start_time:.1f}s")
    
    adjustment_model.eval()
    with torch.no_grad():
        test_image, test_metadata, test_target = train_dataset[0]
        test_image = test_image.unsqueeze(0).to(device)
        test_metadata = test_metadata.unsqueeze(0).to(device)
        
        prediction = adjustment_model(test_image, test_metadata)
        absolute_pred = apply_adjustments(prediction, test_metadata)
        
        print(f"Sample Prediction:")
        print(f"As Shot:   Temp {test_metadata[0,0].item():.0f}K, Tint {test_metadata[0,1].item():.0f}")
        print(f"Predicted: Temp {absolute_pred[0,0].item():.0f}K, Tint {absolute_pred[0,1].item():.0f}")
        print(f"Actual:    Temp {test_target[0].item():.0f}K, Tint {test_target[1].item():.0f}")
        print(f"Temp Error: {abs(absolute_pred[0,0].item() - test_target[0].item()):.0f}K")

print(f"\nTRAINING COMPLETE!")
print(f"Final losses: {[f'{loss:.4f}' for loss in train_losses]}")

if train_losses[-1] < train_losses[0] * 0.8:  
    print("EXCELLENT: Significant learning occurred!")
elif train_losses[-1] < train_losses[0]:
    print("GOOD: Model is learning!")
else:
    print("Model needs more tuning")

In [ ]:
temp_adjustments = []
tint_adjustments = []

for i in range(min(100, len(train_df))):  
    temp_adj = (train_df.iloc[i]['Temperature'] - train_df.iloc[i]['currTemp']) / 1000.0
    tint_adj = (train_df.iloc[i]['Tint'] - train_df.iloc[i]['currTint']) / 10.0
    temp_adjustments.append(temp_adj)
    tint_adjustments.append(tint_adj)

print(f"Temperature adjustments: Min {min(temp_adjustments):.2f}, Max {max(temp_adjustments):.2f}, Mean {np.mean(temp_adjustments):.2f}")
print(f"Tint adjustments: Min {min(tint_adjustments):.2f}, Max {max(tint_adjustments):.2f}, Mean {np.mean(tint_adjustments):.2f}")

class BetterAdjustmentModel(nn.Module):
    def __init__(self):
        super(BetterAdjustmentModel, self).__init__()
        
        self.backbone = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
        self.backbone.classifier = nn.Identity()
        
        for param in list(self.backbone.parameters())[:-50]: 
            param.requires_grad = False
            
        image_feature_dim = 1280
        
        self.metadata_fc = nn.Sequential(
            nn.Linear(8, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU()
        )
        
        self.combined_fc = nn.Sequential(
            nn.Linear(image_feature_dim + 128, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 2)
        )
        
        self.tanh = nn.Tanh()
        
        self.combined_fc[-1].weight.data.normal_(0, 0.01)
        self.combined_fc[-1].bias.data.uniform_(-0.1, 0.1)  
        
    def forward(self, image, metadata):
        image_features = self.backbone(image)
        metadata_features = self.metadata_fc(metadata)
        combined = torch.cat([image_features, metadata_features], dim=1)
        adjustments = self.combined_fc(combined)
        adjustments = self.tanh(adjustments)
        return adjustments

print("\nCreating better model with batch normalization and careful initialization...")
better_model = BetterAdjustmentModel().to(device)
better_optimizer = torch.optim.AdamW(better_model.parameters(), lr=0.0001, weight_decay=1e-5)

print(f"Better model created with {sum(p.numel() for p in better_model.parameters()):,} parameters")
print(f"Trainable parameters: {sum(p.numel() for p in better_model.parameters() if p.requires_grad):,}")

better_model.eval()
with torch.no_grad():
    test_pred = better_model(test_image, test_metadata)
    absolute_pred = apply_adjustments(test_pred, test_metadata)
    print(f"Better model initial prediction: Temp {absolute_pred[0,0].item():.0f}K, Tint {absolute_pred[0,1].item():.0f}")

In [ ]:
print("\nQuick training test with better model...")

better_model.train()
for i in range(3):  # Just 3 batches to see if it learns
    batch_images, batch_metadata, batch_targets = next(iter(train_loader))
    batch_images = batch_images.to(device)
    batch_metadata = batch_metadata.to(device)
    batch_targets = batch_targets.to(device)
    
    better_optimizer.zero_grad()
    predicted_adjustments = better_model(batch_images, batch_metadata)
    loss = adjustment_loss(predicted_adjustments, batch_targets, batch_metadata)
    loss.backward()
    better_optimizer.step()
    
    if i == 0:
        print(f"First batch loss: {loss.item():.4f}")

better_model.eval()
with torch.no_grad():
    test_pred = better_model(test_image, test_metadata)
    absolute_pred = apply_adjustments(test_pred, test_metadata)
    print(f"After quick training: Temp {absolute_pred[0,0].item():.0f}K, Tint {absolute_pred[0,1].item():.0f}")
    print(f"Actual target: Temp {test_target[0].item():.0f}K, Tint {test_target[1].item():.0f}")
    print(f"Temp Error: {abs(absolute_pred[0,0].item() - test_target[0].item()):.0f}K")

In [ ]:
print("TRYING SIMPLER APPROACH: Basic CNN + Metadata")

class SimpleWhiteBalanceModel(nn.Module):
    def __init__(self):
        super(SimpleWhiteBalanceModel, self).__init__()
        
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        
        self.fc = nn.Sequential(
            nn.Linear(128 + 8, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 2)  
        )
        
    def forward(self, image, metadata):
        image_features = self.conv_layers(image)
        image_features = image_features.view(image_features.size(0), -1)
        
        combined = torch.cat([image_features, metadata], dim=1)
        output = self.fc(combined)
        return output

print("Creating simple model...")
simple_model = SimpleWhiteBalanceModel().to(device)
simple_optimizer = torch.optim.Adam(simple_model.parameters(), lr=0.001)

print(f"Simple model created with {sum(p.numel() for p in simple_model.parameters()):,} parameters")

simple_model.eval()
with torch.no_grad():
    simple_transform = transforms.Compose([
        transforms.Resize((128, 128)),  
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    simple_dataset = WhiteBalanceDataset(
        csv_path=os.path.join(dataset_path, 'Train', 'sliders.csv'),
        images_dir=os.path.join(dataset_path, 'Train', 'images'),
        transform=simple_transform,
        is_training=True
    )
    
    simple_image, simple_metadata, simple_target = simple_dataset[0]
    simple_image = simple_image.unsqueeze(0).to(device)
    simple_metadata = simple_metadata.unsqueeze(0).to(device)
    
    simple_pred = simple_model(simple_image, simple_metadata)
    print(f"Simple model prediction: Temp {simple_pred[0,0].item():.0f}K, Tint {simple_pred[0,1].item():.0f}")
    print(f"Actual target: Temp {simple_target[0].item():.0f}K, Tint {simple_target[1].item():.0f}")

In [ ]:
print("\nTRYING CLASSIFICATION APPROACH")

temp_bins = [2000, 3000, 4000, 5000, 6000, 7000, 8000, 10000, 15000, 50000]
tint_bins = [-150, -100, -50, -20, 0, 20, 50, 100, 150]

def values_to_classes(temp_values, tint_values):
    """Convert continuous values to class labels"""
    temp_classes = torch.bucketize(temp_values, torch.tensor(temp_bins))
    tint_classes = torch.bucketize(tint_values, torch.tensor(tint_bins))
    return temp_classes, tint_classes

def classes_to_values(temp_classes, tint_classes):
    """Convert class labels back to approximate values"""
    temp_values = torch.tensor([temp_bins[min(c, len(temp_bins)-1)] for c in temp_classes])
    tint_values = torch.tensor([tint_bins[min(c, len(tint_bins)-1)] for c in tint_classes])
    return temp_values, tint_values

test_temp = torch.tensor([4150.0, 2500.0, 12000.0])
test_tint = torch.tensor([2.0, -30.0, 80.0])

temp_classes, tint_classes = values_to_classes(test_temp, test_tint)
temp_recon, tint_recon = classes_to_values(temp_classes, tint_classes)

print("Classification approach test:")
print(f"   Original: {test_temp.numpy()}, {test_tint.numpy()}")
print(f"   Classes: {temp_classes.numpy()}, {tint_classes.numpy()}")
print(f"   Reconstructed: {temp_recon.numpy()}, {tint_recon.numpy()}")

class ClassificationModel(nn.Module):
    def __init__(self, num_temp_classes=len(temp_bins)+1, num_tint_classes=len(tint_bins)+1):
        super(ClassificationModel, self).__init__()
        
        self.backbone = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
        self.backbone.classifier = nn.Identity()
        
        self.classifier = nn.Sequential(
            nn.Linear(1280 + 8, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, num_temp_classes + num_tint_classes)
        )
        
        self.num_temp_classes = num_temp_classes
        self.num_tint_classes = num_tint_classes
        
    def forward(self, image, metadata):
        image_features = self.backbone(image)
        combined = torch.cat([image_features, metadata], dim=1)
        logits = self.classifier(combined)
        
        temp_logits = logits[:, :self.num_temp_classes]
        tint_logits = logits[:, self.num_temp_classes:]
        
        return temp_logits, tint_logits

print("\nCreating classification model...")
class_model = ClassificationModel().to(device)
class_optimizer = torch.optim.Adam(class_model.parameters(), lr=0.001)

print(f"Classification model created")

In [ ]:
print("TRAINING CLASSIFICATION MODEL")

class ClassificationDataset(Dataset):
    def __init__(self, csv_path, images_dir, transform=None):
        self.data = pd.read_csv(csv_path)
        self.images_dir = images_dir
        self.transform = transform
        
        self.temp_classes, self.tint_classes = values_to_classes(
            torch.tensor(self.data['Temperature'].values),
            torch.tensor(self.data['Tint'].values)
        )
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image_id = row['id_global']
        image_path = os.path.join(self.images_dir, f"{image_id}.tif")
        
        try:
            image = Image.open(image_path).convert('RGB')
        except:
            image = Image.new('RGB', (224, 224), color='gray')
        
        if self.transform:
            image = self.transform(image)
        
        metadata_features = [
            row['currTemp'], row['currTint'], row['aperture'],
            row['focalLength'], row['isoSpeedRating'], row['shutterSpeed'],
            row['flashFired'], row['grayscale']
        ]
        metadata = torch.tensor(metadata_features, dtype=torch.float32)
        
        return image, metadata, self.temp_classes[idx], self.tint_classes[idx]

class_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class_train_dataset = ClassificationDataset(
    csv_path=os.path.join(dataset_path, 'Train', 'sliders.csv'),
    images_dir=os.path.join(dataset_path, 'Train', 'images'),
    transform=class_transform
)

class_train_loader = DataLoader(class_train_dataset, batch_size=16, shuffle=True)

print(f"Classification dataset created: {len(class_train_dataset)} samples")
print(f"Temperature classes: {len(temp_bins)+1} classes")
print(f"Tint classes: {len(tint_bins)+1} classes")

for images, metadata, temp_classes, tint_classes in class_train_loader:
    print(f"Batch test:")
    print(f"- Images: {images.shape}")
    print(f"- Temp classes: {temp_classes[:5].numpy()}")  # Show first 5
    print(f"- Tint classes: {tint_classes[:5].numpy()}")
    break

In [ ]:
import time
from tqdm import tqdm

class_model.train()
criterion = nn.CrossEntropyLoss()
num_epochs = 3

print(f"\nSTARTING CLASSIFICATION TRAINING")

for epoch in range(num_epochs):
    epoch_loss = 0.0
    temp_correct = 0
    tint_correct = 0
    total_samples = 0
    start_time = time.time()
    
    for batch_idx, (images, metadata, temp_targets, tint_targets) in enumerate(tqdm(class_train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")):
        images = images.to(device)
        metadata = metadata.to(device)
        temp_targets = temp_targets.to(device)
        tint_targets = tint_targets.to(device)
        
        class_optimizer.zero_grad()
        temp_logits, tint_logits = class_model(images, metadata)
        
        temp_loss = criterion(temp_logits, temp_targets)
        tint_loss = criterion(tint_logits, tint_targets)
        total_loss = temp_loss + tint_loss
        
        total_loss.backward()
        class_optimizer.step()
        
        epoch_loss += total_loss.item()
        
        _, temp_preds = torch.max(temp_logits, 1)
        _, tint_preds = torch.max(tint_logits, 1)
        temp_correct += (temp_preds == temp_targets).sum().item()
        tint_correct += (tint_preds == tint_targets).sum().item()
        total_samples += temp_targets.size(0)
    
    avg_loss = epoch_loss / len(class_train_loader)
    temp_accuracy = 100.0 * temp_correct / total_samples
    tint_accuracy = 100.0 * tint_correct / total_samples
    
    print(f"Epoch {epoch+1} Complete:")
    print(f"   - Average Loss: {avg_loss:.4f}")
    print(f"   - Temp Accuracy: {temp_accuracy:.1f}%")
    print(f"   - Tint Accuracy: {tint_accuracy:.1f}%")
    print(f"   - Time: {time.time() - start_time:.1f}s")
    
    class_model.eval()
    with torch.no_grad():
        test_image, test_metadata, test_temp_class, test_tint_class = class_train_dataset[0]
        test_image = test_image.unsqueeze(0).to(device)
        test_metadata = test_metadata.unsqueeze(0).to(device)
        
        temp_logits, tint_logits = class_model(test_image, test_metadata)
        temp_pred_class = torch.argmax(temp_logits, 1)
        tint_pred_class = torch.argmax(tint_logits, 1)
        
        temp_pred_value, tint_pred_value = classes_to_values(temp_pred_class.cpu(), tint_pred_class.cpu())
        temp_actual_value, tint_actual_value = classes_to_values(
            torch.tensor([test_temp_class]), torch.tensor([test_tint_class])
        )
        
        print(f"Sample Prediction:")
        print(f"Predicted: Temp {temp_pred_value[0].item():.0f}K, Tint {tint_pred_value[0].item():.0f}")
        print(f"Actual:    Temp {temp_actual_value[0].item():.0f}K, Tint {tint_actual_value[0].item():.0f}")

print(f"\nCLASSIFICATION TRAINING COMPLETE!")

In [ ]:
class_model.eval()
all_temp_preds = []
all_temp_actual = []

with torch.no_grad():
    for i in range(min(50, len(class_train_dataset))):  # Sample 50
        image, metadata, temp_class, tint_class = class_train_dataset[i]
        image = image.unsqueeze(0).to(device)
        metadata = metadata.unsqueeze(0).to(device)
        
        temp_logits, tint_logits = class_model(image, metadata)
        temp_pred_class = torch.argmax(temp_logits, 1)
        
        temp_pred, _ = classes_to_values(temp_pred_class.cpu(), torch.tensor([0]))
        temp_actual, _ = classes_to_values(torch.tensor([temp_class]), torch.tensor([0]))
        
        all_temp_preds.append(temp_pred[0].item())
        all_temp_actual.append(temp_actual[0].item())

print(f"Sample predictions vs actual:")
for i in range(10):
    print(f"  {all_temp_preds[i]:5.0f}K vs {all_temp_actual[i]:5.0f}K")

In [ ]:
print("CREATING SUBMISSION FILE")

validation_dataset = WhiteBalanceDataset(
    csv_path=os.path.join(dataset_path, 'Validation', 'sliders_input.csv'),
    images_dir=os.path.join(dataset_path, 'Validation', 'images'),
    transform=class_transform,
    is_training=False
)

validation_loader = DataLoader(validation_dataset, batch_size=16, shuffle=False)

class_model.eval()
predictions = []
image_ids = []

with torch.no_grad():
    for images, metadata, ids in tqdm(validation_loader, desc="Predicting"):
        images = images.to(device)
        metadata = metadata.to(device)
        
        temp_logits, tint_logits = class_model(images, metadata)
        temp_pred_class = torch.argmax(temp_logits, 1)
        tint_pred_class = torch.argmax(tint_logits, 1)
        
        temp_pred_values, tint_pred_values = classes_to_values(
            temp_pred_class.cpu(), tint_pred_class.cpu()
        )
        
        predictions.extend(zip(temp_pred_values.numpy(), tint_pred_values.numpy()))
        image_ids.extend(ids)

submission_df = pd.DataFrame({
    'id_global': image_ids,
    'Temperature': [int(round(pred[0])) for pred in predictions],
    'Tint': [int(round(pred[1])) for pred in predictions]
})

print(f"Submission shape: {submission_df.shape}")
print(f"Temperature range: {submission_df['Temperature'].min()} to {submission_df['Temperature'].max()}")
print(f"Tint range: {submission_df['Tint'].min()} to {submission_df['Tint'].max()}")

submission_file = 'white_balance_submission.csv'
submission_df.to_csv(submission_file, index=False)
print(f"Submission saved as: {submission_file}")

print("\nSUBMISSION PREVIEW:")
print(submission_df.head(10))